In [ ]:
import cv2
import mediapipe as mp
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def process_video(input_path, output_path, rotate=False):
    cap = cv2.VideoCapture(input_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out    = cv2.VideoWriter(output_path, fourcc, fps, (height if rotate else width, width if rotate else height) if rotate else (width, height))

    print(f"התחלנו לעבד את הווידאו ({total_frames} פריימים)...")

    with mp_pose.Pose(static_image_mode=False,
                      model_complexity=2,
                      enable_segmentation=False,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5) as pose:

        frame_num = 0
        start_time = time.time()

        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            frame_num += 1

            if rotate:
                frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0,0,255), thickness=2))

            progress_text = f"Frame {frame_num}/{total_frames}"
            cv2.putText(frame, progress_text, (30, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

            out.write(frame)
            cv2.imshow('Pose Analysis (press Q to quit)', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        out.release()
        cv2.destroyAllWindows()

        duration = time.time() - start_time
        print(f"השלד שובץ בהצלחה על {frame_num} פריימים.")
        print(f"הסרטון נשמר ב: {output_path}")
        print(f"זמן ריצה כולל: {duration:.1f} שניות.")

# שימוש
process_video(
    input_path='/Users/shay_ziv/Downloads/elawolk.mp4',   #FILE_NAME
    output_path='/Users/shay_ziv/Downloads/pose_output02.mp4',
    rotate=True
)


התחלנו לעבד את הווידאו (1806 פריימים)...


I0000 00:00:1754305195.153318 20942323 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1754305195.250923 20942408 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754305195.303861 20942409 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754305195.375294 20942409 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


השלד שובץ בהצלחה על 1806 פריימים.
הסרטון נשמר ב: /Users/shay_ziv/Downloads/pose_output02.mp4
זמן ריצה כולל: 147.6 שניות.
